## The first model attemped is LSTM. Why? Because I am a sadistic person.

## This section is to be filled out later when I HAVE SOME RESULTS. THAT DAY WILL COME. SOMEDAY.

In [64]:
import music21 as m21

import pandas as pd
import numpy as np

import copy
import cPickle

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

In [4]:
f = open('clean_bach.p','rb')
clean_bach = cPickle.load(f)
f.close()

In [5]:
clean_bach.head()

,BWV,orig_key,beats_per_measure,forward_note,back_note,forward_measure,back_measure,note_in_measure_position,quarter_in_measure_position,soprano_pitch_cat,alto_pitch_cat,tenor_pitch_cat,bass_pitch_cat,soprano_pitch_cont,alto_pitch_cont,tenor_pitch_cont,bass_pitch_cont
bwv421_0,bwv421,a minor,4,0,159,0,10,1,4,G4,D4,B-3,G2,1,-4,-8,-23
bwv421_1,bwv421,a minor,4,1,158,0,10,2,4,G4,D4,B-3,G2,1,-4,-8,-23
bwv421_2,bwv421,a minor,4,2,157,0,10,3,4,G4,D4,C4,A2,1,-4,-6,-21
bwv421_3,bwv421,a minor,4,3,156,0,10,4,4,G4,D4,C4,A2,1,-4,-6,-21
bwv421_4,bwv421,a minor,4,4,155,1,9,1,1,G4,D4,D4,B-2,1,-4,-4,-20


## Creating target and predictor variables
#### Create a master y and X for each categorical and continuous
### KEEP THESE AS DATAFRAMES

### Categorical Version

#### target y variable

In [9]:
# a series
master_y_cat = clean_bach['soprano_pitch_cat']

In [53]:
master_y_cat.value_counts()

G4      7330
A4      7169
D5      6706
C5      4883
B4      4026
B-4     3538
E5      2369
G5      1867
D4      1238
F#5     1111
F4       962
F#4      939
A5       678
F5       656
E4       595
Rest     528
E-5      476
E-4      362
B5       327
C#5      268
C4       192
B-5      159
C6        87
B-3       78
G#4       50
D6        30
D#5       16
D#4       12
C#4        8
G#5        8
B3         8
Name: soprano_pitch_cat, dtype: int64

In [55]:
n_y_cat_variables = master_y_cat.value_counts().shape[0]

#### target y variable AS DUMMY VARIABLES

In [79]:
master_y_cat_dummy = pd.get_dummies(master_y_cat)

In [80]:
master_y_cat_dummy.head()

,A4,A5,B-3,B-4,B-5,B3,B4,B5,C#4,C#5,...,E5,F#4,F#5,F4,F5,G#4,G#5,G4,G5,Rest
bwv421_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
bwv421_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
bwv421_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
bwv421_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
bwv421_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### predictor X variables

In [7]:
clean_bach.columns

Index([u'BWV', u'orig_key', u'beats_per_measure', u'forward_note',
       u'back_note', u'forward_measure', u'back_measure',
       u'note_in_measure_position', u'quarter_in_measure_position',
       u'soprano_pitch_cat', u'alto_pitch_cat', u'tenor_pitch_cat',
       u'bass_pitch_cat', u'soprano_pitch_cont', u'alto_pitch_cont',
       u'tenor_pitch_cont', u'bass_pitch_cont'],
      dtype='object')

In [10]:
master_X_cat = clean_bach[[u'beats_per_measure', u'forward_note',
       u'back_note', u'forward_measure', u'back_measure',
       u'note_in_measure_position', u'quarter_in_measure_position',
       u'soprano_pitch_cat', u'alto_pitch_cat', u'tenor_pitch_cat',
       u'bass_pitch_cat']]

#### predictor X variables as dummy variables

In [82]:
parts_df = master_X_cat[['soprano_pitch_cat', 'alto_pitch_cat', 
                         u'tenor_pitch_cat', u'bass_pitch_cat']]

In [83]:
parts_df_dummy = pd.get_dummies(parts_df)

In [85]:
parts_df_dummy.head(2)

,soprano_pitch_cat_A4,soprano_pitch_cat_A5,soprano_pitch_cat_B-3,soprano_pitch_cat_B-4,soprano_pitch_cat_B-5,soprano_pitch_cat_B3,soprano_pitch_cat_B4,soprano_pitch_cat_B5,soprano_pitch_cat_C#4,soprano_pitch_cat_C#5,...,bass_pitch_cat_F#4,bass_pitch_cat_F2,bass_pitch_cat_F3,bass_pitch_cat_F4,bass_pitch_cat_G#2,bass_pitch_cat_G#3,bass_pitch_cat_G2,bass_pitch_cat_G3,bass_pitch_cat_G4,bass_pitch_cat_Rest
bwv421_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
bwv421_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [86]:
master_X_cat_dummies = pd.concat(
    [master_X_cat[[u'beats_per_measure', u'forward_note',u'back_note', 
                  u'forward_measure', u'back_measure',
                  u'note_in_measure_position', 
                  u'quarter_in_measure_position']],
    parts_df_dummy],
    axis=1)

In [87]:
master_X_cat_dummies.head()

,beats_per_measure,forward_note,back_note,forward_measure,back_measure,note_in_measure_position,quarter_in_measure_position,soprano_pitch_cat_A4,soprano_pitch_cat_A5,soprano_pitch_cat_B-3,...,bass_pitch_cat_F#4,bass_pitch_cat_F2,bass_pitch_cat_F3,bass_pitch_cat_F4,bass_pitch_cat_G#2,bass_pitch_cat_G#3,bass_pitch_cat_G2,bass_pitch_cat_G3,bass_pitch_cat_G4,bass_pitch_cat_Rest
bwv421_0,4,0,159,0,10,1,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
bwv421_1,4,1,158,0,10,2,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
bwv421_2,4,2,157,0,10,3,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bwv421_3,4,3,156,0,10,4,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bwv421_4,4,4,155,1,9,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
list(master_X_cat_dummies.columns)

['beats_per_measure',
 'forward_note',
 'back_note',
 'forward_measure',
 'back_measure',
 'note_in_measure_position',
 'quarter_in_measure_position',
 'soprano_pitch_cat_A4',
 'soprano_pitch_cat_A5',
 'soprano_pitch_cat_B-3',
 'soprano_pitch_cat_B-4',
 'soprano_pitch_cat_B-5',
 'soprano_pitch_cat_B3',
 'soprano_pitch_cat_B4',
 'soprano_pitch_cat_B5',
 'soprano_pitch_cat_C#4',
 'soprano_pitch_cat_C#5',
 'soprano_pitch_cat_C4',
 'soprano_pitch_cat_C5',
 'soprano_pitch_cat_C6',
 'soprano_pitch_cat_D#4',
 'soprano_pitch_cat_D#5',
 'soprano_pitch_cat_D4',
 'soprano_pitch_cat_D5',
 'soprano_pitch_cat_D6',
 'soprano_pitch_cat_E-4',
 'soprano_pitch_cat_E-5',
 'soprano_pitch_cat_E4',
 'soprano_pitch_cat_E5',
 'soprano_pitch_cat_F#4',
 'soprano_pitch_cat_F#5',
 'soprano_pitch_cat_F4',
 'soprano_pitch_cat_F5',
 'soprano_pitch_cat_G#4',
 'soprano_pitch_cat_G#5',
 'soprano_pitch_cat_G4',
 'soprano_pitch_cat_G5',
 'soprano_pitch_cat_Rest',
 'alto_pitch_cat_A#4',
 'alto_pitch_cat_A-3',
 'alto_pitch_

### Continuous Version

In [11]:
# a series
master_y_cont = clean_bach['soprano_pitch_cont']

In [12]:
master_X_cont = clean_bach[[u'beats_per_measure', u'forward_note',
       u'back_note', u'forward_measure', u'back_measure',
       u'note_in_measure_position', u'quarter_in_measure_position',
       u'soprano_pitch_cont', u'alto_pitch_cont', u'tenor_pitch_cont',
       u'bass_pitch_cont']]

# Create a function that transforms your model results back into music21 data. 

## CATEGORICAL

In [ ]:
# first write a function that takes the dummy variables and turns them
# back into 

# LSTM Model

### Model 1
- variables:

    - soprano

    - alto
    
    - tenor
    
    - bass
    
    - no other ones
    
- categorical or continuous:

    - categorical

- train test split

    - 30%
    
- variables as features or time lag? time lag as features or time lag?

    - variables as features; time lag as time lag
    
- normalize/standardize?

    - no
    
- time lagged: 

    - NO
    
- input nodes:

    - no
    
- intermediate nodes:

    - 10 LSTM
    
- activation:

    - softmax
    
- optimizer:

    - adam
    
- number epochs:

    - 10
    
- number batch size

    - 20

- dropout

    - NA
    
- CNN

    - NA


- split out train test set 30%
- define data set - X and y variables
- reshape X

In [81]:
# create y var for model 1; make it an array
y1 = master_y_cat_dummy.values

In [97]:
# create X vars for model 1; make it an array
columns_not_needed = ['beats_per_measure', 'forward_note',
                     'back_note', 'forward_measure', 'back_measure',
                     'note_in_measure_position',
                      'quarter_in_measure_position']

columns_needed = [x for x in master_X_cat_dummies.columns if x not in columns_not_needed]
X1 = master_X_cat_dummies[columns_needed].values
X1.shape

(46676, 141)

In [98]:
# train test split
# 70% of the number of rows
train_end_index = int(round(y1.shape[0] * 0.7))

y1_train, y1_test = y1[:train_end_index], y1[train_end_index:]
X1_train, X1_test = X1[:train_end_index], X1[train_end_index:]

In [99]:
X1_train[0:10]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [100]:
y1_train.shape, y1_test.shape, X1_train.shape, X1_test.shape

((32673, 31), (14003, 31), (32673, 141), (14003, 141))

In [101]:
# set variables
time_lag = 1 
# actually it is 0 since we are not doing time series, but for the 
# purpose of the 3D X matrix we will make it 1
n_features = X1.shape[1]

In [102]:
# reshape X to be [samples, time steps, features]
X1_train_RS = np.reshape(X1_train, (X1_train.shape[0], 
                                       time_lag, 
                                      n_features))

X1_test_RS = np.reshape(X1_test, (X1_test.shape[0], 
                                       time_lag, 
                                      n_features))

In [104]:
X1_train_RS[0:2]

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  0.,  0.,  0.,  0.,  

In [105]:
model = Sequential()
model.add(LSTM(10, input_dim=141))
model.add(Dense(n_y_cat_variables, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [106]:
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_4 (LSTM)                    (None, 10)            6080        lstm_input_4[0][0]               
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 31)            341         lstm_4[0][0]                     
Total params: 6421
____________________________________________________________________________________________________
None


In [107]:
model.fit(X1_train_RS, y1_train, nb_epoch=10, batch_size=20, verbose=2)

Epoch 1/10
20s - loss: 1.0051 - acc: 0.7992
Epoch 2/10
19s - loss: 0.0746 - acc: 0.9951
Epoch 3/10
20s - loss: 0.0280 - acc: 0.9976
Epoch 4/10
20s - loss: 0.0139 - acc: 0.9982
Epoch 5/10
19s - loss: 0.0076 - acc: 0.9991
Epoch 6/10
21s - loss: 0.0046 - acc: 0.9993
Epoch 7/10
23s - loss: 0.0030 - acc: 0.9995
Epoch 8/10
20s - loss: 0.0021 - acc: 0.9997
Epoch 9/10
22s - loss: 0.0014 - acc: 0.9998
Epoch 10/10
22s - loss: 9.3088e-04 - acc: 1.0000


In [108]:
predictions_prob = model.predict_proba(X1_train_RS)

32640/32673 [============================>.] - ETA: 0s

In [110]:
predictions_prob[0:15]

array([[  2.70059175e-08,   9.97016400e-07,   1.82116686e-08,
          1.28576517e-06,   1.26351072e-08,   1.89386853e-07,
          3.40691759e-06,   4.71739314e-11,   1.49100671e-07,
          9.07438903e-07,   4.95727932e-07,   2.45824964e-07,
          1.24633750e-11,   6.46876401e-08,   5.39172689e-08,
          1.67110372e-07,   5.49933521e-09,   1.08094766e-09,
          2.18606445e-07,   4.19309032e-10,   9.27433859e-07,
          2.76907794e-08,   1.01254136e-06,   9.37660758e-08,
          1.20734171e-06,   1.63785412e-08,   1.38655139e-07,
          1.83888478e-08,   9.99980927e-01,   1.24188948e-09,
          7.38228573e-06],
       [  2.70059175e-08,   9.97016400e-07,   1.82116686e-08,
          1.28576517e-06,   1.26351072e-08,   1.89386853e-07,
          3.40691759e-06,   4.71739314e-11,   1.49100671e-07,
          9.07438903e-07,   4.95727932e-07,   2.45824964e-07,
          1.24633750e-11,   6.46876401e-08,   5.39172689e-08,
          1.67110372e-07,   5.49933521e-09,

In [114]:
predictions = model.predict_classes(X1_train_RS)

32640/32673 [============================>.] - ETA: 0s

In [115]:
predictions[0:15]

array([28, 28, 28, 28, 28, 28,  0,  0,  3,  3,  3,  3,  0,  0,  0])

### Model 2

In [ ]:
# create X and y based on time lag requested (
# (modified from kiefer lab)
def create_XY(timeseries, lag=1):
    ts = timeseries.ravel()
    y = ts[lag:]
    X = []
    for i in range(lag):
        current_pred = ts[i:]
        current_pred = current_pred[:len(y)]
        X.append(current_pred)
    return np.array(X).T, y

### Using MCMC

### K Nearest Neighbors Model